In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_json('recipes.json', lines=True)

In [3]:
df.head()

,rate,author_tip,budget,prep_time,ingredients,name,author,difficulty,people_quantity,cook_time,tags,total_time,image,nb_comments
0,5,,bon marché,15 min,"[600g de pâte à crêpe, 1/2 orange, 1/2 banane,...",6 ingrédients que l’on peut ajouter sur une cr...,Nutella,très facile,6,10 min,"[Crêpe, Crêpes sucrées, Végétarien, Dessert]",25 min,https://assets.afcdn.com/recipe/20171006/72810...,1
1,3,,Coût moyen,2 h,"[1kg d'agneau, 300g de riz, 24 abricots secs, ...",Agneau à l'abricot (recette turque),oscar_15167769,Niveau moyen,6,1 h,[Plat principal],3 h,,1
2,4.6,,bon marché,15 min,"[1kg d'épaule d'agneau désossée, 3 oignons, 2 ...",Agneau à l'oriental,zoreille,très facile,6,1 h,"[Sans gluten, Plat principal]",1h15,,10
3,5,,bon marché,45 min,"[2 aubergines, 3 tomates, 700g d'épaule d'agne...",Agneau à la grec (aubergine & feta),richard_148,facile,4,1 h,"[Sans gluten, Plat principal]",1h45,,6
4,5,,Coût moyen,30 min,"[1 gigot d'agneau, 2 oignons, 6gousses d'ail, ...",Agneau à la mangue et aux amandes,roxane_16871304,très facile,4,1 h,"[Sans gluten, Plat principal]",1h30,,2


In [4]:
df['name'] = df['name'].str.capitalize()

In [5]:
df['difficulty'].value_counts()

facile          4796
très facile     3942
Niveau moyen     719
difficile         34
Name: difficulty, dtype: int64

In [6]:
df['difficulty'] = df['difficulty'].str.lower()

In [7]:
df['budget'].value_counts()

bon marché    5907
Coût moyen    3256
assez cher     328
Name: budget, dtype: int64

In [8]:
df['budget'] = df['budget'].str.lower()

In [9]:
(df['image'].values != '').sum()

4085

In [10]:
(df['author_tip'].values != '').sum()

0

In [11]:
df['rate'].value_counts()

       2202
5      1511
4       975
4.8     710
4.7     647
4.5     645
4.3     488
4.6     388
4.4     365
4.9     320
3       263
4.2     169
3.5     164
2        97
3.8      95
3.7      88
4.1      81
3.3      55
1        50
3.9      39
3.6      31
2.5      29
3.4      20
3.2      16
0        11
2.8      11
2.7       7
2.3       7
3.1       2
2.6       2
2.4       1
1.5       1
1.7       1
Name: rate, dtype: int64

In [12]:
df['rate'] = df['rate'].replace('', np.nan, regex=True)
df['rate'] = df['rate'].astype(float)
df['rate'].value_counts()

5.0    1511
4.0     975
4.8     710
4.7     647
4.5     645
4.3     488
4.6     388
4.4     365
4.9     320
3.0     263
4.2     169
3.5     164
2.0      97
3.8      95
3.7      88
4.1      81
3.3      55
1.0      50
3.9      39
3.6      31
2.5      29
3.4      20
3.2      16
2.8      11
0.0      11
2.3       7
2.7       7
2.6       2
3.1       2
1.5       1
2.4       1
1.7       1
Name: rate, dtype: int64

In [13]:
df['people_quantity'].value_counts()

4      5256
6      1824
2      1319
8       320
3       226
5       211
1       176
10       65
12       33
20       12
          8
24        7
7         5
15        4
30        4
40        4
14        3
9         3
16        3
25        3
500       2
100       1
50        1
18        1
Name: people_quantity, dtype: int64

In [14]:
df['people_quantity'] = df['people_quantity'].replace('', 0, regex=True)
df['people_quantity'] = df['people_quantity'].astype(int)
df['people_quantity'].value_counts()

4      5256
6      1824
2      1319
8       320
3       226
5       211
1       176
10       65
12       33
20       12
0         8
24        7
7         5
40        4
15        4
30        4
25        3
9         3
14        3
16        3
500       2
18        1
100       1
50        1
Name: people_quantity, dtype: int64

In [15]:
from datetime import timedelta
import re

UNITS = {'m':'minutes', 'h':'hours', 'j':'days'}

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

def timeparse(s):
    s = s.replace('min', 'm').replace(' ', '')
    return int(timedelta(**{
        UNITS.get(m.group('unit').lower(), 'minutes'): float(m.group('val'))
        for m in re.finditer(r'(?P<val>\d+(\.\d+)?)(?P<unit>[minhj]?)', s, flags=re.I)
    }).total_seconds()/60)

df['prep_time_in_minutes'] = df['prep_time'].apply(timeparse)
df['cook_time_in_minutes'] = df['cook_time'].apply(timeparse)
df['total_time_in_minutes'] = df['total_time'].apply(timeparse)

df.groupby('total_time').agg({'total_time_in_minutes': ['max']})

,total_time_in_minutes
,max
total_time,
,0
1 h,60
1 j,1440
1 j 1 h 30 min,1530
1 j 11 h 20 min,2120
...,...
7h30,450
7h35,455


In [16]:
df['prep_time_in_minutes'] = df['prep_time_in_minutes'].replace('', 0, regex=True)
df['prep_time_in_minutes'] = df['prep_time_in_minutes'].astype(int)

df['cook_time_in_minutes'] = df['cook_time_in_minutes'].replace('', 0, regex=True)
df['cook_time_in_minutes'] = df['cook_time_in_minutes'].astype(int)

df['total_time_in_minutes'] = df['total_time_in_minutes'].replace('', 0, regex=True)
df['total_time_in_minutes'] = df['total_time_in_minutes'].astype(int)

df['total_time_in_minutes'].value_counts()

40     781
30     760
50     759
45     703
60     697
      ... 
67       1
123      1
131      1
435      1
863      1
Name: total_time_in_minutes, Length: 185, dtype: int64

In [17]:
df.explode('ingredients')['ingredients'].value_counts()

Sel                                                          5519
Poivre                                                       5135
1 oignon                                                     1775
Huile d'olive                                                1239
2 oignons                                                     838
                                                             ... 
7tranches de poitrine fumée                                     1
4 langoustines                                                  1
1boîte de tomate en dés (ou 4 tomates fraiches en saison)       1
1 oignon rouge idéalement                                       1
1verre de vinaigre blanc                                        1
Name: ingredients, Length: 27867, dtype: int64

In [18]:
df.explode('tags')['tags'].value_counts()

Plat principal         9292
Sans gluten            4424
Viande                 1909
Végétarien             1642
Pâtes, riz, semoule    1358
                       ... 
Pain perdu                1
Mille-feuille             1
Noël / Tribu              1
Autres gâteaux            1
Vegan                     1
Name: tags, Length: 445, dtype: int64

In [19]:
df['author'].value_counts()

Anonyme              633
Marmiton_Recettes    104
Herta                 87
Nutella               67
colorado              61
                    ... 
julia_17558185         1
laura_16612526         1
agathe_14909315        1
caramel                1
emma_14090057          1
Name: author, Length: 6399, dtype: int64

In [20]:
def to_lower(s):
    return s.lower()

df['budget'] = df['budget'].apply(to_lower)
df['difficulty'] = df['difficulty'].apply(to_lower)

In [21]:
df['number_of_comments'] = df['nb_comments']
df['id'] = df.index + 1
df = df[['id','name','rate','people_quantity','prep_time_in_minutes','cook_time_in_minutes','total_time_in_minutes',
         'image','number_of_comments','author','difficulty','budget','ingredients','tags']]

In [22]:
df = df.replace('', np.nan, regex=True)

In [23]:
df.head()

,id,name,rate,people_quantity,prep_time_in_minutes,cook_time_in_minutes,total_time_in_minutes,image,number_of_comments,author,difficulty,budget,ingredients,tags
0,1,6 ingrédients que l’on peut ajouter sur une cr...,5.0,6,15,10,25,https://assets.afcdn.com/recipe/20171006/72810...,1,Nutella,très facile,bon marché,"[600g de pâte à crêpe, 1/2 orange, 1/2 banane,...","[Crêpe, Crêpes sucrées, Végétarien, Dessert]"
1,2,Agneau à l'abricot (recette turque),3.0,6,120,60,180,NaN,1,oscar_15167769,niveau moyen,coût moyen,"[1kg d'agneau, 300g de riz, 24 abricots secs, ...",[Plat principal]
2,3,Agneau à l'oriental,4.6,6,15,60,75,NaN,10,zoreille,très facile,bon marché,"[1kg d'épaule d'agneau désossée, 3 oignons, 2 ...","[Sans gluten, Plat principal]"
3,4,Agneau à la grec (aubergine & feta),5.0,4,45,60,105,NaN,6,richard_148,facile,bon marché,"[2 aubergines, 3 tomates, 700g d'épaule d'agne...","[Sans gluten, Plat principal]"
4,5,Agneau à la mangue et aux amandes,5.0,4,30,60,90,NaN,2,roxane_16871304,très facile,coût moyen,"[1 gigot d'agneau, 2 oignons, 6gousses d'ail, ...","[Sans gluten, Plat principal]"


In [24]:
if not os.path.exists('tables'):
    os.makedirs('tables')

In [25]:
df = df.assign(author_id=df.author.factorize()[0] + 1)
df_authors = df[['author_id', 'author']].drop_duplicates()
df_authors.columns = ['id', 'name']
df_authors.to_csv('tables/authors.csv', index=False, encoding='utf-8')

In [26]:
df = df.assign(difficulty_id=df.difficulty.factorize()[0] + 1)
df_difficulties = df[['difficulty_id', 'difficulty']].drop_duplicates()
df_difficulties.columns = ['id', 'name']
df_difficulties.to_csv('tables/difficulties.csv', index=False, encoding='utf-8')

In [27]:
df = df.assign(budget_id=df.budget.factorize()[0] + 1)
df_budgets = df[['budget_id', 'budget']].drop_duplicates()
df_budgets.columns = ['id', 'name']
df_budgets.to_csv('tables/budgets.csv', index=False, encoding='utf-8')

In [28]:
df_recipes = df[['id','name','rate','people_quantity','prep_time_in_minutes','cook_time_in_minutes','total_time_in_minutes', 
                 'image','number_of_comments','author_id','difficulty_id','budget_id']]
df_recipes.to_csv('tables/recipes.csv', index=False, encoding='utf-8')

In [29]:
df_recipes_ingredients = df.explode('ingredients')
df_recipes_ingredients = df_recipes_ingredients.assign(ingredient_id=df_recipes_ingredients.ingredients.factorize()[0] + 1)

df_ingredients = df_recipes_ingredients[['ingredient_id', 'ingredients']].drop_duplicates()
df_ingredients.columns = ['id', 'name']
df_ingredients.to_csv('tables/ingredients.csv', index=False, encoding='utf-8')

df_recipes_ingredients = df_recipes_ingredients[['id', 'ingredient_id']]
df_recipes_ingredients.columns = ['recipe_id', 'ingredient_id']
df_recipes_ingredients.to_csv('tables/recipes_ingredients.csv', index=False, encoding='utf-8')

In [30]:
df_recipes_tags = df.explode('tags')
df_recipes_tags = df_recipes_tags.assign(tag_id=df_recipes_tags.tags.factorize()[0] + 1)

df_tags = df_recipes_tags[['tag_id', 'tags']].drop_duplicates()
df_tags.columns = ['id', 'name']
df_tags.to_csv('tables/tags.csv', index=False, encoding='utf-8')

df_recipes_tags = df_recipes_tags[['id', 'tag_id']]
df_recipes_tags.columns = ['recipe_id', 'tag_id']
df_recipes_tags.to_csv('tables/recipes_tags.csv', index=False, encoding='utf-8')